In [ ]:
1. 자극 키워드 목록 설정
2. W 수치가 있는 게시글과 댓글 데이터활용 
포포함된 경우 → 해당 키워드들의 K 중 가장 큰 값 사용
여러 키워드가 겹치면 max(K_i) 또는 mean(K_i) 적용
3. 수식 적용
# D 모델
import pandas as pd

df = pd.read_excel('C:/Users/speec/OneDrive/Desktop/딥테크팁스/tips/output/Poc2_1.xlsx')
df.head()
def clean_text(text):
    import re
    text = re.sub(r"http\S+", "", text)  # URL 제거
    text = re.sub(r"[^\uAC00-\uD7A3a-zA-Z0-9\s]", "", text)  # 특수문자 제거 (한글/영문/숫자만 남김)
    text = text.lower()
    return text.strip()

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.tag import pos_tag

def extract_keywords(text):
    # 텍스트 토큰화
    tokens = word_tokenize(text)
    
    # 품사 태깅
    tagged = pos_tag(tokens)
    
    # 명사만 추출 (NN: 명사, NNS: 복수명사, NNP: 고유명사, NNPS: 복수고유명사)
    nouns = [word for word, tag in tagged if tag.startswith('NN')]
    
    # 단어 길이 2 이상 필터링
    nouns = [n for n in nouns if len(n) > 1]
    
    return nouns

# D output 파일 읽기

import pandas as pd

# 부정 어휘 사전 정의
negative_lexicon = [
    "해킹", "유출", "피해", "사고", "사건", "공격", "침해", "위험",
    "불안", "불신", "불편", "문제", "대란", "먹통", "복제", "탈취", 
    "도난", "사기", "의혹", "접속대기중", "무용지물", "부실", "늑장",
    "손해", "혼란", "충격", "비판", "비난", "무책임", "잘못", "신세",
    "방랑", "구걸", "신고", "대처실패", "책임전가", "보상", "과징금",
    "유심복제", "유심탈취", "불똥", "유심해킹", "정보유출", "부정확",
    "지연", "우려", "접속불가", "알림없음", "모르면손해", "이상현상",
    "터지다", "금융사기", "먹통현상", "불통", "불능", "재시작", "조치없음"
]

# CSV 파일 읽기
df = pd.read_csv('C:/Users/speec/OneDrive/Desktop/딥테크팁스/tips/output/Doutput.csv')

# merged_tokens에서 부정 어휘 카운트
df['negative_count'] = df['merged_tokens'].apply(lambda x: sum(1 for token in eval(x) if token in negative_lexicon))

df.head()

# D output 파일 읽기

import pandas as pd

# 부정 어휘 사전 정의
negative_lexicon = [
    "해킹", "유출", "피해", "사고", "사건", "공격", "침해", "위험",
    "불안", "불신", "불편", "문제", "대란", "먹통", "복제", "탈취", 
    "도난", "사기", "의혹", "접속대기중", "무용지물", "부실", "늑장",
    "손해", "혼란", "충격", "비판", "비난", "무책임", "잘못", "신세",
    "방랑", "구걸", "신고", "대처실패", "책임전가", "보상", "과징금",
    "유심복제", "유심탈취", "불똥", "유심해킹", "정보유출", "부정확",
    "지연", "우려", "접속불가", "알림없음", "모르면손해", "이상현상",
    "터지다", "금융사기", "먹통현상", "불통", "불능", "재시작", "조치없음"
]

# CSV 파일 읽기
df = pd.read_csv('C:/Users/speec/OneDrive/Desktop/딥테크팁스/tips/output/Doutput.csv')

# merged_tokens에서 부정 어휘 카운트 및 단어 추출
def count_negative_words(tokens):
    neg_words = [token for token in eval(tokens) if token in negative_lexicon]
    return len(neg_words), neg_words

# 부정 단어 개수와 단어 목록 추출
df['negative_count'], df['negative_words'] = zip(*df['merged_tokens'].apply(count_negative_words))

# 각 행의 부정 단어와 그 개수를 딕셔너리 형태로 저장
df['negative_word_counts'] = df['negative_words'].apply(lambda x: dict(pd.Series(x).value_counts()))

df.head()

lwp_df = pd.read_excel(r'C:\Users\speec\OneDrive\Desktop\딥테크팁스\tips\LWP_model_results.xlsx')
lwp_df.head(1)
df['doc_str'] 
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

# 1. 부정 키워드를 문자열로 연결
df['doc_str'] = df['negative_words'].apply(lambda lst: ' '.join(lst))

# 2. 6시간 단위로 그룹화하여 문서 생성
grouped_docs = df.groupby('time_block')['doc_str'].apply(lambda x: ' '.join(x)).reset_index()

# 3. TF-IDF 벡터라이저로 변환
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(grouped_docs['doc_str'])

# 4. 키워드 이름 추출
keywords = vectorizer.get_feature_names_out()

# 5. TF-IDF 결과를 데이터프레임으로 변환
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=keywords)
tfidf_df['time_block'] = grouped_docs['time_block']

# 6. 상위 5개 TF-IDF 평균을 D-score로 정의
top_k = 3
tfidf_df['D_score'] = tfidf_df[keywords].apply(lambda row: sorted(row, reverse=True)[:top_k], axis=1).apply(lambda x: sum(x)/top_k)

# 결과 확인
display(tfidf_df[['time_block', 'D_score']])

# 7. time_block과 D_score만 포함된 데이터프레임을 엑셀로 저장
result_df = tfidf_df[['time_block', 'D_score']]
result_df.to_excel('C:/Users/speec/OneDrive/Desktop/딥테크팁스/tips/output/D_score_results.xlsx', index=False)

lwp_df.head()
# Convert time_block to datetime and format to show only up to hours
result_df['time_block'] = pd.to_datetime(result_df['time_block']).dt.strftime('%Y-%m-%d %H:00')

# Save to df20 and display
df20 = result_df.copy()
display(df20)


df20.head(1)
# Convert date_hour to datetime and format to show only up to hours
lwp_df['date_hour'] = pd.to_datetime(lwp_df['date_hour']).dt.strftime('%Y-%m-%d %H:00')

# Save to df21 and display
df21 = lwp_df.copy()
df21.head()
# Merge df20 and df21 on date_hour and time_block columns
merged_df = pd.merge(df20, df21, left_on='time_block', right_on='date_hour', how='inner').dropna()

# Save merged dataframe as LWPD model to Excel
merged_df.drop('date_hour', axis=1).to_excel('C:/Users/speec/OneDrive/Desktop/딥테크팁스/tips/output/LWPD_model.xlsx', index=False)



merged_df .columns



# Drop date_hour column since it's identical to time_block
merged_df = merged_df.drop('date_hour', axis=1)
merged_df.info()
